In [1]:
import pandas as pd
import numpy as np
import math

from tqdm import tqdm
import time

C:\Users\Rusla\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Rusla\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Rusla\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
RANDOM_SEED = 13022022

In [3]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  1614 non-null   object
dtypes: object(1)
memory usage: 12.7+ KB


In [5]:
test.sample(3)

,Smiles
1285,CCCOc1ccc(/C=C2/SC(=O)NC2=O)cc1
581,O=C(O)CCCNC(=O)c1cccnc1
749,CN1CCN(CCCCN2C(=O)CN(/N=C/c3ccc(-c4ccc(Cl)cc4)...


In [6]:
train = pd.read_csv('Task/train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.0+ KB


In [8]:
train.sample(3)

,Smiles,Active
2884,CC(O)CC(=O)O,False
1266,OCc1c[nH]cn1,False
953,O=C(O)c1ccc(S(=O)(=O)N(Cc2ccc(OC(F)(F)F)cc2)c2...,False


In [9]:
list(train['Active'].unique())

[False, True]

In [10]:
list_active = {False: 0, True: 1}

In [11]:
train['Active'] = train['Active'].map(list_active)

In [12]:
train.sample()

,Smiles,Active
68,CCCCCCOc1nsnc1C1=CCCN(C)C1.O=C(O)C(=O)O,0


In [13]:
train['train'] = 1
test['train'] = 0

In [14]:
df = pd.concat([train, test], ignore_index=True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7171 entries, 0 to 7170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Smiles  7171 non-null   object 
 1   Active  5557 non-null   float64
 2   train   7171 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 168.2+ KB


In [16]:
df['train'].unique()

array([1, 0], dtype=int64)

In [17]:
df[df['train'] == 1]['Active'].value_counts()

0.0    5351
1.0     206
Name: Active, dtype: int64

In [18]:
from collections import Counter

atoms = Counter()
for i in df['Smiles'].unique():
    for j in i:
        atoms[j] += 1

In [19]:
len(atoms.keys())

45

In [20]:
cont = 0
for i in atoms.keys():
    new_value = len(atoms.keys()) - cont
    atoms[i] = new_value
    cont += 1

In [21]:
atoms

Counter({'C': 45,
         'O': 44,
         'c': 43,
         '1': 42,
         '2': 41,
         '[': 40,
         'n': 39,
         'H': 38,
         ']': 37,
         '(': 36,
         'N': 35,
         ')': 34,
         '@': 33,
         '.': 32,
         'l': 31,
         '=': 30,
         'S': 29,
         '3': 28,
         '4': 27,
         'F': 26,
         '-': 25,
         's': 24,
         '/': 23,
         '5': 22,
         'o': 21,
         'a': 20,
         '+': 19,
         '#': 18,
         'I': 17,
         'P': 16,
         'B': 15,
         'r': 14,
         '\\': 13,
         'Z': 12,
         '6': 11,
         '7': 10,
         '8': 9,
         'e': 8,
         'A': 7,
         'K': 6,
         'M': 5,
         'g': 4,
         'i': 3,
         'L': 2,
         '9': 1})

In [22]:
df['len'] = df['Smiles'].apply(lambda x: len(x))

In [23]:
df['1'] = df['Smiles'].apply(lambda x: list(x))

In [24]:
df.sample(3)

,Smiles,Active,train,len,1
1161,CC(CC(C(N)=O)(c1ccccc1)c1ccccc1)N(C)C.O=S(=O)(O)O,0.0,1,49,"[C, C, (, C, C, (, C, (, N, ), =, O, ), (, c, ..."
868,Cc1cc(C(=O)Nc2cc(Oc3ccc4nc(NC(=O)C5CC5)cn4n3)c...,0.0,1,57,"[C, c, 1, c, c, (, C, (, =, O, ), N, c, 2, c, ..."
3274,COc1ccc(Cl)cc1NC(=O)Nc1cc(C)nc2ccccc12,0.0,1,38,"[C, O, c, 1, c, c, c, (, C, l, ), c, c, 1, N, ..."


In [25]:
df['len'].unique().max()

707

In [ ]:
new_df = pd.DataFrame()
for i in tqdm(range(len(df['1']))):
    e = pd.DataFrame(df.loc[i].explode())
    e.reset_index(drop=True, inplace=True)
    e = e.T
    new_df = pd.concat([new_df, e], ignore_index = True)

 61%|███████████████████████████████████████████████▋                              | 4384/7171 [02:54<04:12, 11.05it/s]

In [ ]:
new_df.sample(5)

In [ ]:
new_df.fillna(0, inplace = True)

In [ ]:
new_df.rename(columns = {0: 'Smiles', 1: 'Active', 2: 'train', 3: 'len'}, inplace = True)
new_df = new_df.astype({"Active": "Int64"})

In [ ]:
new_df.set_index(['Smiles', 'Active', 'train', 'len'], inplace = True)

In [ ]:
new_df.sample(5)

In [ ]:
from tqdm import tqdm
import time

for i in tqdm(range(len(new_df[4]))):
    new_df.iloc[i] = new_df.iloc[i].map(atoms)

In [ ]:
new_df.sample(5)

In [ ]:
new_df.reset_index(inplace = True)
new_df.set_index(['Smiles'], inplace = True)

In [ ]:
new_df.sample()

In [ ]:
#new_df = new_df.astype({"Active": "Int64"})

In [ ]:
train_new = new_df[new_df['train'] == 1]
test_new =new_df[new_df['train'] == 0]

In [ ]:
X = train_new.drop(['Active', 'train'], axis = 1)
y = train_new['Active']

In [ ]:
X.info()